<h3> Extracting protein sequences' features using ProtBert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers

     |████████████████████████████████| 2.9 MB 30.1 MB/s 
     |████████████████████████████████| 596 kB 40.3 MB/s 
     |████████████████████████████████| 3.3 MB 34.2 MB/s 
     |████████████████████████████████| 895 kB 37.5 MB/s 
     |████████████████████████████████| 56 kB 3.3 MB/s 


In [ ]:
import torch
from transformers import BertModel, BertTokenizer
import re
import os
import requests
from tqdm.auto import tqdm
import pandas as pd
import numpy as np

<b>2. Load the vocabulary and ProtBert Model</b>

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)

Downloading:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/361 [00:00<?, ?B/s]

In [ ]:

model = BertModel.from_pretrained("Rostlab/prot_bert")

Downloading:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')
device

device(type='cpu')

In [ ]:
model = model.to(device)
model = model.eval()

In [ ]:

dataset = pd.read_csv(r"/content/drive/MyDrive/peptides-bayes - peptides.csv")
dataset


,seq,class
0,EQGE,0
1,KTEE,0
2,TLKA,0
3,ARFAEVY,0
4,KTTLLSEQ,0
...,...,...
1922,MAKKSLIAREKKRKKLEEKFYLIRRYPTKEMSKGGSLSESWEIQGK...,1
1923,MAKKSLIQREKKRQNLEKKYKILRNSLKKKITETSSLDEKWEFQKK...,1
1924,MARKSLIQREKKRQKLEQKYHSIRRSSKKEISKVPSLSDKWEIYGK...,1
1925,MATLADPRDIILAPVISEKSYGLLDDNVYTFLVRPDSNKTQIKIAV...,1


In [ ]:
num_1 = 1000
num_2 = len(dataset)+1
dataset1 = dataset.iloc[num_1:num_2]
print(len(dataset1))
dataset1.tail(3)

927


,seq,class
1924,MARKSLIQREKKRQKLEQKYHSIRRSSKKEISKVPSLSDKWEIYGK...,1
1925,MATLADPRDIILAPVISEKSYGLLDDNVYTFLVRPDSNKTQIKIAV...,1
1926,MFYFISRKFYDQFKYGILTDKTNKLLKNNVYTFDVDIQMSKRQFKD...,1


In [ ]:
#dataset1.tail(3)

In [ ]:
sequences_Example = list(dataset1["seq"])
len(sequences_Example)
sequences_Example
sequence=[]
for i in sequences_Example:
  j = re.sub("([A-Z])", " \\1", i).strip()
  sequence.append(j)
sequence

['D E K P K L I L P T P A P P N L P Q L V G G G G G N R K D G F G V S V D A H Q K V W T S D N G G H S I G V S P G Y S Q H L P G P Y G N S R P D Y R I G A G Y S Y N F',
 'D E K P K L I L P T P A P P N L P Q L V G G G G G N R K D G F G V S V D A H Q K V W T S D N G R H S I G V T P G Y S Q H L G G P Y G N S R P D Y R I G A G Y S Y N F',
 'M D V V I S I A V V L G G V T T G L V N L V K S M D V V S P K Y L P L V S L A I G M V F G L A M S P L L G I T L Y V G A I S G L V A G L S A M G F Y E L T K T P A E',
 'M Q M G E S M A K I V K F V Y F V I I F A S P F V V A N H E I S G W I T E L P F G M C T S I L D C P M D S C T H P Q Q P W C E L H G V P I L Y H G S E I G L C I C I',
 'M Q P T I M E E L T K E Q V L E E L G A Y K Q I I A S A A T R D R G E L L R T V I E A V A R P R P R P C I R A G G Y C N I L N V C C A G L T C E E H D I Q D A V C V',
 'D D M T M K P T P P P Q Y P L N L Q G G G G G Q S G D G F G F A V Q G H Q K V W T S D N G R H E I G L N G G Y G Q H L G G P Y G N S E P S W K V G S T Y T Y R 

In [ ]:
#sequences_Example = ["A A C S D R A H G H I C E S F K S F C K D S G R N G V K L R A N C K K T C G L C ",
#                     "A A E F P D F Y D S E E Q M G P H Q E A E D E K D R A D Q R V L T E E E K K E L E N L A A M D L E L Q K I A E K F S Q R "]

In [ ]:



#sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

In [ ]:
#sequences_Example


<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [ ]:
ids = tokenizer.batch_encode_plus(sequence[:], add_special_tokens=True, pad_to_max_length=True)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2217: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)

In [ ]:
len(input_ids[0])

102

In [ ]:
attention_mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 1, 1, 0],
        [1, 1, 1,  ..., 1, 1, 1]])

<b>6. Extracting sequences' features and load it into the CPU if needed<b>

In [ ]:
with torch.no_grad():
  embedding = model(input_ids=input_ids,attention_mask=attention_mask)[0]
embedding

tensor([[[ 9.9389e-03,  4.7547e-02,  9.0957e-02,  ..., -7.1630e-02,
           3.2087e-02,  3.2170e-02],
         [ 6.3967e-02,  7.4811e-02,  1.1846e-01,  ...,  3.6516e-03,
           6.8370e-02, -7.0137e-02],
         [ 1.3322e-01, -2.9294e-02,  2.1643e-02,  ..., -1.1722e-02,
          -1.0257e-01,  1.0252e-01],
         ...,
         [-2.4304e-02, -4.0743e-02, -6.3574e-02,  ...,  7.0419e-02,
           4.0440e-02,  2.8484e-02],
         [ 5.6088e-03, -1.5899e-01, -1.4577e-01,  ..., -6.5970e-02,
           6.6667e-02,  8.8628e-02],
         [ 1.8334e-03, -9.7953e-02, -1.1461e-01,  ..., -7.7863e-02,
           7.3321e-03,  4.3297e-02]],

        [[ 1.6689e-02,  4.4679e-02,  8.4919e-02,  ..., -6.5114e-02,
           7.6500e-03,  4.0132e-02],
         [ 5.9304e-02,  9.0768e-02,  9.5811e-02,  ...,  1.0707e-02,
           6.9989e-02, -6.6968e-02],
         [ 1.2626e-01, -2.3846e-02,  2.5410e-02,  ..., -1.4153e-02,
          -1.0883e-01,  1.0007e-01],
         ...,
         [-2.0183e-02, -5

In [ ]:
embedding = embedding.cpu().numpy()
embedding.shape

(927, 102, 1024)

In [ ]:
dictt = dict()
for point_num in range(len(embedding)):
    cls_token_vector = list(embedding[point_num][0])
    dictt[point_num] = cls_token_vector
print(dictt.get((49)))

[0.03494927, 0.0010927027, 0.06419576, -0.14884408, 0.1449477, -0.030517824, -0.08556632, -0.009235804, -0.037937645, -0.08587874, -0.032128405, -0.052793242, 0.12317126, 0.045973767, -0.010536882, 0.06157065, -0.022952091, 0.041755777, -0.0021690375, -0.016552903, 0.00013588725, -0.019492498, -0.041942198, -0.057253603, -0.056477048, 0.06438275, 0.022230895, 0.012006682, 0.021001518, -0.044585913, -0.010601775, -0.017831942, -0.099633366, 0.120289125, -0.0018516924, -0.07564099, 0.032357804, -0.085415944, -0.07034103, -0.0661167, -0.0016880601, 0.17981964, -0.03395924, -0.017531408, -0.019136496, 0.016751954, 0.10740041, 0.020933675, 0.026076451, -0.0014604665, -0.009048849, 0.024299439, -0.009877678, -0.045864392, -0.06979826, -0.06336054, -0.07696025, -0.030345319, -0.066841245, -0.05388552, -0.04766249, 0.0012823667, 0.117261656, 0.032034628, 0.040638313, -0.010224514, 0.1250238, 0.008192428, -0.056656543, 0.028444955, -0.05361252, 0.06941128, -0.040795047, 0.0027778582, 0.01921727

In [ ]:
columns_list = [(str("Feature")+str(num+1)) for num in range(len(dictt.get(0)))]
bert_dataframe = pd.DataFrame()

for key in list(dictt.keys()):
    bert_dataframe = bert_dataframe.append(pd.Series(dictt.get(key), index=columns_list), ignore_index=True)
print(len(bert_dataframe))
bert_dataframe.head(3)

927


,Feature1,Feature10,Feature100,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature101,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature102,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,Feature103,Feature104,Feature105,Feature106,Feature107,Feature108,Feature109,Feature11,Feature110,Feature111,...,Feature963,Feature964,Feature965,Feature966,Feature967,Feature968,Feature969,Feature97,Feature970,Feature971,Feature972,Feature973,Feature974,Feature975,Feature976,Feature977,Feature978,Feature979,Feature98,Feature980,Feature981,Feature982,Feature983,Feature984,Feature985,Feature986,Feature987,Feature988,Feature989,Feature99,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999
0,0.009939,-0.066409,0.016250,-0.005664,0.114986,-0.052833,-0.047260,-0.066393,-0.010551,-0.061278,-0.060052,0.028840,-0.016081,0.057962,-0.003933,-0.028201,-0.015819,0.004044,0.004648,-0.057900,-0.001496,-0.101040,-0.086032,-0.018076,-0.159949,-0.157848,-0.071022,-0.071630,0.032087,0.032170,-0.001284,0.097237,0.037029,0.051405,0.012621,-0.029758,-0.097465,-0.079078,0.121953,0.076340,...,0.101911,-0.091049,-0.046062,0.004467,0.004229,-0.014121,-0.060950,-0.064173,0.129896,0.005255,-0.037874,-0.005980,-0.043500,0.031993,0.023259,-0.031540,0.074449,0.089603,-0.035776,0.012459,-0.105954,0.082218,-0.139309,-0.026311,0.018853,-0.043661,-0.054128,-0.028426,0.032052,0.035786,-0.158291,-0.027342,-0.054663,0.020818,-0.029832,-0.015204,0.016733,-0.077800,0.002664,0.011959
1,0.016689,-0.068242,0.013779,-0.024844,0.104987,-0.049246,-0.058283,-0.065186,-0.016076,-0.073583,-0.068587,0.038908,-0.024336,0.052768,0.003971,-0.016552,-0.013636,-0.013911,0.006351,-0.057117,0.009065,-0.084489,-0.081585,-0.013041,-0.147160,-0.151648,-0.077203,-0.065114,0.007650,0.040132,-0.000445,0.096935,0.030268,0.040273,0.015055,-0.041939,-0.083621,-0.084833,0.118201,0.064274,...,0.105435,-0.079407,-0.054356,0.000029,0.010424,-0.011266,-0.071980,-0.068160,0.131946,0.005014,-0.037821,0.002168,-0.061354,0.056807,0.018407,-0.018317,0.076959,0.080381,-0.041658,-0.011120,-0.108423,0.084276,-0.136980,-0.029375,0.026430,-0.033749,-0.056896,-0.027079,0.013345,0.045432,-0.157048,0.000585,-0.058568,0.012364,-0.005431,-0.014122,0.028091,-0.077883,0.006975,0.007202
2,0.052136,-0.058846,-0.012990,0.043054,-0.019665,-0.008616,-0.072305,0.019289,0.026620,-0.018891,-0.002772,0.073048,-0.065702,0.084897,0.051094,-0.097873,-0.025816,-0.044002,0.050180,0.008373,0.014059,-0.047337,-0.047891,0.019909,-0.107804,0.051959,-0.010381,-0.005446,-0.048839,0.078949,0.045618,0.060736,-0.037022,-0.022917,-0.015760,-0.092436,-0.019112,-0.042829,0.105413,0.035805,...,0.080859,-0.115474,-0.053647,0.008990,0.023070,-0.005717,0.016183,0.057314,0.042840,0.000893,-0.059476,-0.010601,-0.034354,0.007981,0.027767,0.036892,-0.018740,-0.016406,-0.091422,-0.009069,-0.055957,-0.006634,-0.094313,0.028726,-0.028577,0.006438,0.055133,-0.091071,0.008527,0.022823,0.047143,0.025482,-0.049120,-0.005072,0.024238,-0.083550,0.016736,0.045691,0.040172,-0.058286


In [ ]:
len(bert_dataframe.columns)

1024

In [ ]:
final_bert_dataframe = pd.DataFrame()
for i in range(len(bert_dataframe.columns)):
    column = list(bert_dataframe["Feature"+str(i+1)])
    final_bert_dataframe["Feature"+str(i+1)] = column
print(len(final_bert_dataframe))
final_bert_dataframe.head(3)

927


,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature985,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024
0,0.009939,0.047547,0.090957,-0.138436,0.148956,-0.010896,-0.084781,0.065114,0.017134,-0.066409,-0.079078,-0.026440,0.052610,0.028148,-0.019957,0.015839,-0.095701,0.066310,0.019431,-0.075389,0.004717,0.027693,0.055799,-0.114115,-0.107973,-0.057721,0.085902,-0.016368,-0.035011,-0.054856,-0.100488,-0.031667,-0.041413,-0.001004,0.118519,0.016690,0.094820,-0.062614,-0.005992,-0.004851,...,0.018853,-0.043661,-0.054128,-0.028426,0.032052,-0.158291,-0.027342,-0.054663,0.020818,-0.029832,-0.015204,0.016733,-0.077800,0.002664,0.011959,-0.005664,0.114986,-0.052833,-0.047260,-0.066393,-0.010551,-0.061278,-0.060052,0.028840,-0.016081,-0.003933,-0.028201,-0.015819,0.004044,0.004648,-0.057900,-0.001496,-0.101040,-0.086032,-0.018076,-0.157848,-0.071022,-0.071630,0.032087,0.032170
1,0.016689,0.044679,0.084919,-0.130764,0.168424,-0.023065,-0.104082,0.064288,0.017827,-0.068242,-0.084833,-0.016277,0.060551,0.026589,-0.045105,0.030098,-0.097938,0.067586,0.024733,-0.071260,0.016730,0.051793,0.059810,-0.108775,-0.084220,-0.054506,0.079500,-0.020293,-0.030284,-0.070972,-0.086387,-0.039938,-0.042448,0.017474,0.127991,0.013203,0.096349,-0.063176,-0.008376,-0.010144,...,0.026430,-0.033749,-0.056896,-0.027079,0.013345,-0.157048,0.000585,-0.058568,0.012364,-0.005431,-0.014122,0.028091,-0.077883,0.006975,0.007202,-0.024844,0.104987,-0.049246,-0.058283,-0.065186,-0.016076,-0.073583,-0.068587,0.038908,-0.024336,0.003971,-0.016552,-0.013636,-0.013911,0.006351,-0.057117,0.009065,-0.084489,-0.081585,-0.013041,-0.151648,-0.077203,-0.065114,0.007650,0.040132
2,0.052136,-0.021984,0.039147,-0.137069,0.126188,-0.018822,-0.055218,-0.015919,0.040944,-0.058846,-0.042829,-0.024865,0.074456,0.008870,-0.064161,0.059834,-0.005776,0.056018,-0.012272,0.006611,0.011558,-0.035673,0.024780,-0.019043,-0.084618,0.085291,-0.003507,0.005372,0.002209,-0.070279,-0.041011,0.025137,-0.122748,0.076889,-0.014918,-0.020697,0.083028,-0.085044,-0.028087,-0.017713,...,-0.028577,0.006438,0.055133,-0.091071,0.008527,0.047143,0.025482,-0.049120,-0.005072,0.024238,-0.083550,0.016736,0.045691,0.040172,-0.058286,0.043054,-0.019665,-0.008616,-0.072305,0.019289,0.026620,-0.018891,-0.002772,0.073048,-0.065702,0.051094,-0.097873,-0.025816,-0.044002,0.050180,0.008373,0.014059,-0.047337,-0.047891,0.019909,0.051959,-0.010381,-0.005446,-0.048839,0.078949


In [ ]:
final_bert_dataframe.to_csv("bert_features13.csv", encoding='utf-8', index=False)

In [ ]:
np.save('bert_dataframe3' +'.csv', bert_dataframe)

In [ ]:
!cp "bert_features13.csv" "/content/drive/MyDrive/Bayeslabs"

cp: cannot stat 'bert_features13.csv': No such file or directory


In [ ]:
df = pd.concat(map(pd.read_csv,['/content/drive/MyDrive/bert features/bert_features11.csv','/content/drive/MyDrive/bert features/bert_features12.csv','/content/drive/MyDrive/bert features/bert_features13.csv']), ignore_index=True)
print(df)

      Feature1  Feature2  Feature3  ...  Feature1022  Feature1023  Feature1024
0    -0.104880  0.139020 -0.087928  ...    -0.094516    -0.058622     0.033227
1     0.128913  0.098447  0.000031  ...     0.008098    -0.025453     0.007406
2     0.173506  0.132104  0.052542  ...    -0.066719    -0.061537     0.005588
3     0.136856  0.114502  0.044591  ...    -0.013837    -0.105768    -0.010426
4     0.117469  0.155314  0.051438  ...     0.007174    -0.034558    -0.004036
...        ...       ...       ...  ...          ...          ...          ...
1922  0.023276  0.001638  0.021983  ...     0.014082    -0.007467     0.044222
1923  0.046298  0.003432  0.040643  ...    -0.003969    -0.002994     0.046555
1924  0.038750  0.012081  0.015797  ...    -0.001982    -0.008875     0.042219
1925  0.114387  0.024027  0.055476  ...    -0.038954    -0.067874     0.082245
1926  0.163595  0.032631  0.098400  ...    -0.055791    -0.080113     0.054746

[1927 rows x 1024 columns]


In [ ]:
df.to_csv("final_bert_features.csv", encoding='utf-8', index=False)

In [ ]:
np.save('final_bert_dataframe' +'.csv', bert_dataframe)

In [ ]:
!cp "bert_features13.csv" "/content/drive/MyDrive/Bert features"

cp: cannot stat 'bert_features13.csv': No such file or directory


In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 925 kB 5.1 MB/s 
     |████████████████████████████████| 125 kB 56.6 MB/s 
     |████████████████████████████████| 596 kB 42.5 MB/s 
     |████████████████████████████████| 282 kB 71.3 MB/s 
     |████████████████████████████████| 829 kB 47.6 MB/s 
     |████████████████████████████████| 1.3 MB 56.7 MB/s 
     |████████████████████████████████| 271 kB 59.3 MB/s 
     |████████████████████████████████| 160 kB 56.8 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=7f66c90dd8eb7cfc1945585627e255648875ee81514546ccd5481a518d02995f
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

df = pd.read_csv(r"/content/drive/MyDrive/bert features/final_bert_features.csv")
x = df[[df.loc[:,(df.dtypes=='object').values]]]
dataset = pd.read_csv(r"/content/drive/MyDrive/peptides-bayes - peptides.csv")
y= dataset['class'].values
y
df['class'] = y.tolist()
df['Feature1'].values
X = df.iloc[x=[]]
Y = df['class']
Y
X = preprocessing.StandardScaler().fit(X).transform(X)
#dont bother

In [ ]:
df['class']
df.to_csv("final_bert.csv", encoding='utf-8', index=False)

In [ ]:
df1 = pd.read_csv(r"/content/drive/MyDrive/final_bert.csv")
df1['class']

0       0
1       0
2       0
3       0
4       0
       ..
1922    1
1923    1
1924    1
1925    1
1926    1
Name: class, Length: 1927, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df1, test_size=0.15)
test.shape

(290, 1025)

In [ ]:
train.index = np.arange(1,len(train)+1)
test.index = np.arange(1,len(test)+1)
test

In [ ]:
train.to_csv("final_bert_train.csv", encoding='utf-8', index=False)
test.to_csv("final_bert_test.csv", encoding='utf-8', index=False)

In [ ]:
for train_index, test_index in sss.split(X, Y):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]
  #dont bother

In [ ]:
from torch import nn
from torch.nn.functional import relu 
import torch.nn.functional as F
from torch import tensor
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch
import pandas as pd
import numpy as np
#from pytorch_lightning.metrics.functional import accuracy
from torchmetrics.functional.classification.accuracy import accuracy

class TorchModel(nn.Module):
    def __init__(self, input_size=1024, hidden_size_1=64,hidden_size_2=128,
                 hidden_size_3=64,hidden_size_4=32,hidden_size_5= 32,
                 #hidden_size_5= 32,hidden_size_6 = 32,hidden_size_7 = 16,
                 output_size=2):
        super(TorchModel, self).__init__()
        self.layer_1 = torch.nn.Linear(input_size,hidden_size_1)
        self.layer_2 = torch.nn.Linear(hidden_size_1, hidden_size_2)
        self.layer_3 = torch.nn.Linear(hidden_size_2, hidden_size_3)
        self.layer_4 = torch.nn.Linear(hidden_size_3, hidden_size_4)
        self.layer_5 = torch.nn.Linear(hidden_size_4, hidden_size_5)
        #self.layer_6 = torch.nn.Linear(hidden_size_5, hidden_size_6)
        #self.layer_7 = torch.nn.Linear(hidden_size_6, hidden_size_7)
        self.output_layer = torch.nn.Linear(hidden_size_5, output_size)
       
    def forward(self,input_data):
        output_of_layer_1 = relu(self.layer_1(input_data))
        output_of_layer_2 = relu(self.layer_2(output_of_layer_1))
        output_of_layer_3 = relu(self.layer_3(output_of_layer_2))
        output_of_layer_4 = relu(self.layer_4(output_of_layer_3))
        output_of_layer_5 = relu(self.layer_5(output_of_layer_4))
        #output_of_layer_6 = relu(self.layer_6(output_of_layer_5))
        #output_of_layer_7 = relu(self.layer_7(output_of_layer_6))
        output = self.output_layer(output_of_layer_5)
        final_output = F.log_softmax(output,dim = 1)
        return final_output


class LightningModel(pl.LightningModule):
    def __init__(self):
        super(LightningModel, self).__init__()
        self.model = TorchModel()
        self.batch_size = 32
        #self.loss = nn.
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=0.001)

    def cross_entropy_loss(self, output, labels):
        return torch.nn.NLLLoss()(output, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        output = self.model.forward(x)
        train_loss = self.cross_entropy_loss(output, y)
        #predicted_labels = torch.argmax(output, dim=1)
        train_acc = accuracy(torch.argmax(output, dim=1), y)
        #pbar = {"train_acc":acc}
        self.log("train_performance", {"train_acc":train_acc, "train_loss": train_loss},on_step=False, on_epoch=True, prog_bar=True, logger=True)
        #self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return train_loss
    
    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        output = self.model.forward(x)
        test_loss = self.cross_entropy_loss(output, y)
        test_acc = accuracy(torch.argmax(output, dim=1), y)
        self.log("test_performance", {"test_acc":test_acc, "test_loss": test_loss},on_step=True, on_epoch=True, prog_bar=True, logger=True)
        #output = dict({'test_loss': test_loss,'test_acc': test_acc})


        return test_loss


    def prepare_data(self):
        train_data = pd.read_csv(r"/content/final_bert_train.csv")
        train_labels = train_data["class"]
        train_data = train_data.drop("class" , axis = 1)
        train_data = torch.tensor(np.array(train_data), dtype=torch.float)
        train_=torch.LongTensor(np.array(train_labels))
        self.train_data = []
        for i in range(len(train_data)):
            self.train_data.append((train_data[i], train_labels[i]))
        # self.trn_data = (train_data, train_labels)


        test_data = pd.read_csv(r"/content/final_bert_test.csv")
        test_labels = test_data["class"]
        test_data = test_data.drop("class" , axis = 1)
        test_data = (torch.tensor(np.array(test_data), dtype=torch.float))
        test_labels=torch.LongTensor(np.array(test_labels))
        self.test_data = []
        for i in range(len(test_data)):
            self.test_data.append((test_data[i], test_labels[i]))


    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size = self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size = self.batch_size)


trainer = pl.Trainer(max_epochs=150)
model = LightningModel()
trainer.fit(model)
trainer.test(model,dataloaders = model.test_dataloader())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | TorchModel | 85.4 K
-------------------------------------
85.4 K    Trainable params
0         Non-trainable params
85.4 K    Total params
0.342     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_performance': {'test_acc': tensor(0.9000), 'test_loss': tensor(0.8377)},
 'test_performance_epoch': {'test_acc': tensor(0.9000),
                            'test_loss': tensor(0.8377)}}
--------------------------------------------------------------------------------


[{'test_performance': {'test_acc': tensor(0.9000),
   'test_loss': tensor(0.8377)},
  'test_performance_epoch': {'test_acc': tensor(0.9000),
   'test_loss': tensor(0.8377)}}]

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv(r"/content/drive/MyDrive/Bayes labs/dataframe.csv")
print("Total data points we have in our Dataset is :",len(data))
data.head(3)

Total data points we have in our Dataset is : 13045


,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,labels
0,0.069630,0.037715,0.094908,-0.204495,0.216357,-0.067500,-0.129693,0.017480,0.021069,-0.086695,-0.038282,-0.004839,0.094959,0.033578,-0.061521,0.021535,-0.077589,0.088890,0.063067,-0.089765,-0.006309,0.040383,-0.001253,-0.167778,-0.032887,0.031695,0.069038,0.002625,0.009161,-0.091343,-0.081996,-0.053690,-0.129325,0.088868,0.090851,-0.073844,0.145810,-0.100443,-0.070930,-0.011890,...,0.043572,-0.095108,-0.108759,0.024416,-0.089847,-0.015661,-0.107151,-0.006303,0.003388,0.025380,0.001922,-0.024166,-0.007612,-0.077760,0.027432,0.134993,0.020873,-0.064566,-0.092742,-0.031703,-0.062025,-0.001946,0.072060,-0.021253,-0.000871,-0.091445,0.071739,-0.012103,0.013119,-0.053159,0.060784,-0.099561,-0.142787,0.036709,-0.097751,-0.061995,-0.051084,-0.038646,-0.020350,non_Anti_microbial
1,0.158570,0.027016,0.084353,-0.261657,0.255672,-0.028979,-0.060672,0.003051,-0.072140,-0.145537,0.003898,0.022706,0.045013,0.127011,0.063842,-0.138697,-0.048299,0.098664,-0.026465,-0.073534,0.085967,0.080135,-0.108955,-0.152588,0.037963,-0.013081,0.093290,0.013057,0.058086,-0.135192,0.000354,0.136811,-0.193744,0.033134,0.083954,-0.091634,0.068099,0.003499,-0.105327,-0.154782,...,0.005787,-0.082780,-0.009048,0.046565,-0.040207,-0.089750,-0.103536,0.090986,-0.072441,0.074443,0.065187,-0.018859,0.063093,-0.122281,0.054790,0.119621,-0.117958,-0.008106,-0.071025,0.019424,-0.026795,-0.065976,-0.045075,-0.033450,0.004089,-0.091568,0.090431,0.037361,0.209537,-0.030037,-0.083185,-0.073892,-0.107537,0.053009,-0.203266,-0.116658,-0.052270,-0.067016,0.067302,non_Anti_microbial
2,0.111323,0.024840,0.122577,-0.200168,0.211283,-0.039343,-0.109409,0.027931,-0.040814,-0.111279,-0.036637,-0.010607,0.046858,0.062787,-0.046101,-0.027495,-0.069926,0.055665,0.011457,-0.077800,0.052512,0.011069,-0.041876,-0.195665,-0.018367,0.050151,0.093020,0.008231,0.008631,-0.097777,-0.039131,0.053937,-0.143924,0.058895,0.053202,-0.087114,0.087996,-0.057728,-0.081531,-0.049334,...,0.046643,-0.073014,-0.068133,0.090216,-0.050809,-0.036625,-0.086690,0.051850,0.001070,0.044721,-0.008755,-0.021717,0.047530,-0.103937,0.038788,0.063038,-0.065605,-0.035414,-0.062052,-0.000771,-0.018625,-0.052999,0.018849,-0.025234,-0.004726,-0.055147,0.102750,-0.021590,0.067670,-0.016135,0.022117,-0.088876,-0.142062,0.005176,-0.166395,-0.042158,-0.090205,-0.038277,0.002191,non_Anti_microbial


In [ ]:
data.value_counts("labels")

labels
non_Anti_microbial    9777
Anti_microbial        3268
dtype: int64

In [ ]:
labels = data["labels"]
data = data.drop("labels" , axis = 1)
print("Shape of data is :",data.shape,"Shape of labels is : ",labels.shape)

Shape of data is : (13045, 1024) Shape of labels is :  (13045,)


In [ ]:
from sklearn.model_selection import train_test_split
train_features,test_features,train_labels,test_labels = train_test_split(data, labels, test_size = 0.13, stratify=labels,random_state = 42)
print("Shape of train_features and train labels is :",train_features.shape,train_labels.shape)
print("Shape of test_features and test_labels is : ",test_features.shape,test_labels.shape)

Shape of train_features and train labels is : (11349, 1024) (11349,)
Shape of test_features and test_labels is :  (1696, 1024) (1696,)


In [ ]:
train_features["labels"] = list(train_labels)
test_features["labels"] = list(test_labels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
type(train_labels)

pandas.core.series.Series

In [ ]:
train_features.head(3)

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,labels
2255,0.031776,-0.027921,-0.002877,-0.147194,0.150376,-0.001528,-0.064426,-0.047460,-0.005686,-0.110137,-0.098233,-0.021952,0.007496,-0.077937,0.001968,-0.015003,-0.041122,0.084479,0.057136,0.036204,0.057517,-0.022898,0.027018,-0.104637,-0.156715,-0.009038,0.041330,-0.003780,-0.039153,-0.076083,-0.095316,0.064067,-0.100659,-0.012934,0.073428,-0.048179,0.166126,-0.043250,-0.036603,0.006181,...,0.041017,-0.012835,0.016975,0.008138,-0.098891,-0.046668,-0.076522,0.082843,-0.029855,0.045958,-0.018832,-0.039814,0.107620,-0.057539,-0.061097,-0.027217,-0.023102,-0.026213,-0.083684,-0.040919,-0.025093,-0.058228,0.038443,-0.012139,0.006184,-0.035568,-0.008107,-0.082168,0.001727,-0.090185,-0.002516,-0.028781,-0.114518,-0.005395,-0.057053,-0.059022,-0.067482,-0.016724,0.002891,Anti_microbial
5716,0.135872,0.014606,0.076611,-0.295001,0.277320,-0.032924,-0.051026,-0.013421,-0.085678,-0.137909,0.020649,0.038876,0.039154,0.134754,0.066811,-0.152788,-0.058336,0.105534,-0.040341,-0.079608,0.068903,0.088370,-0.117220,-0.132376,0.034054,-0.009841,0.062385,0.030015,0.070556,-0.140085,0.008696,0.137880,-0.165311,0.010079,0.098351,-0.069681,0.043736,0.001666,-0.122446,-0.161463,...,-0.002398,-0.077538,0.011734,0.047796,-0.055281,-0.104313,-0.097418,0.086787,-0.093739,0.071810,0.065398,-0.006039,0.062626,-0.098479,0.061696,0.124932,-0.117388,0.021206,-0.067915,0.047174,-0.021232,-0.074441,-0.056596,-0.025545,0.005097,-0.097408,0.090363,0.062502,0.224509,-0.004505,-0.082303,-0.055964,-0.123756,0.053434,-0.203528,-0.116633,-0.043016,-0.081660,0.071516,non_Anti_microbial
6978,0.069585,0.009713,0.058913,-0.180797,0.195681,0.012911,-0.106612,0.057295,-0.020182,-0.081798,-0.060896,-0.046798,0.074334,0.090443,-0.009775,0.032702,-0.024203,0.062925,0.076867,-0.102591,0.040713,0.054539,-0.025717,-0.184562,-0.063081,-0.005208,0.044689,-0.034102,0.037094,-0.079286,-0.052217,0.011512,-0.106528,0.081739,0.071955,-0.131061,0.114489,-0.073059,-0.095921,-0.012154,...,0.006437,-0.110682,-0.099758,0.030435,-0.079064,-0.039414,-0.075532,-0.003134,-0.025852,0.076593,0.008769,-0.007426,0.031173,-0.051602,0.065502,0.094191,0.024978,-0.062388,-0.084072,-0.045510,-0.078724,-0.019195,0.061779,-0.001777,0.007160,-0.097733,0.089607,-0.030114,0.005077,-0.057750,-0.004976,-0.122645,-0.190197,-0.018957,-0.112847,-0.076487,-0.033950,-0.019898,0.021438,non_Anti_microbial


In [ ]:
test_features.head(3)

,Feature1,Feature2,Feature3,Feature4,Feature5,Feature6,Feature7,Feature8,Feature9,Feature10,Feature11,Feature12,Feature13,Feature14,Feature15,Feature16,Feature17,Feature18,Feature19,Feature20,Feature21,Feature22,Feature23,Feature24,Feature25,Feature26,Feature27,Feature28,Feature29,Feature30,Feature31,Feature32,Feature33,Feature34,Feature35,Feature36,Feature37,Feature38,Feature39,Feature40,...,Feature986,Feature987,Feature988,Feature989,Feature990,Feature991,Feature992,Feature993,Feature994,Feature995,Feature996,Feature997,Feature998,Feature999,Feature1000,Feature1001,Feature1002,Feature1003,Feature1004,Feature1005,Feature1006,Feature1007,Feature1008,Feature1009,Feature1010,Feature1011,Feature1012,Feature1013,Feature1014,Feature1015,Feature1016,Feature1017,Feature1018,Feature1019,Feature1020,Feature1021,Feature1022,Feature1023,Feature1024,labels
1690,0.109431,0.037502,0.059286,-0.212406,0.184761,-0.002387,-0.094443,-0.014094,-0.041196,-0.110593,0.023350,-0.002643,0.081095,0.111027,-0.002186,-0.043364,-0.025856,0.069302,0.017455,-0.037943,0.024812,0.029315,-0.106831,-0.113370,-0.063722,0.039092,0.029709,0.014165,0.056934,-0.089000,-0.013783,0.076565,-0.155654,0.089465,0.058875,-0.084667,0.042516,-0.025731,-0.108964,-0.149267,...,-0.013593,-0.052182,-0.058593,0.063475,0.032207,-0.052440,-0.028907,-0.010147,-0.077112,0.036872,0.025252,0.020158,0.035512,-0.116798,0.092976,0.078440,-0.007102,-0.060508,-0.082755,0.013825,-0.096522,-0.067320,0.025822,-0.029683,0.019120,-0.078460,0.108477,0.024281,0.147555,0.051941,-0.066832,-0.029229,-0.105289,0.083165,-0.097971,-0.097137,-0.023382,-0.123162,0.075368,non_Anti_microbial
5905,-0.008494,0.027247,-0.012789,-0.137007,0.175489,-0.046696,-0.078308,-0.009429,-0.004535,-0.101400,-0.076042,-0.028133,0.017943,0.001078,-0.001259,-0.000867,-0.027447,0.056188,0.022451,0.028725,0.021201,-0.026593,0.012847,-0.085048,-0.066358,-0.031304,0.017349,-0.054906,-0.043239,-0.134178,-0.078860,0.004786,-0.104230,0.038747,0.125431,-0.061640,0.153986,-0.027879,-0.036898,0.003091,...,0.029707,-0.019854,0.011796,-0.042385,-0.119466,-0.019402,-0.077659,0.093463,-0.006995,0.065490,-0.016651,-0.088461,0.095529,-0.033680,-0.010749,0.007390,-0.080337,-0.057983,-0.217866,-0.065110,-0.088622,-0.034897,0.011049,-0.003211,0.034794,-0.010049,-0.007819,-0.011402,0.064271,-0.049518,0.039848,-0.009495,-0.129781,-0.021887,-0.044729,-0.035315,-0.081538,-0.003337,-0.027138,Anti_microbial
4576,0.054417,0.094189,0.103464,-0.175528,0.163843,-0.040624,-0.144607,0.046408,-0.037799,-0.081449,-0.090296,-0.086181,0.072488,0.020609,-0.073911,0.134762,-0.121659,0.049699,0.090068,-0.019201,0.003705,0.051190,0.026391,-0.116375,-0.005507,-0.001336,0.099146,0.080520,-0.064099,-0.157723,-0.021806,-0.003806,-0.189323,-0.044014,0.040237,-0.001729,0.172977,-0.106001,0.066516,0.007342,...,0.032812,0.012422,-0.018214,-0.022703,-0.128591,0.010230,-0.117096,-0.056186,0.040505,0.032570,0.004417,-0.053004,0.028572,0.063831,0.051129,0.042063,-0.062931,-0.066595,-0.002475,-0.029575,-0.049054,-0.022394,0.027545,0.034095,0.038732,-0.018791,0.009082,-0.009632,-0.046492,-0.077531,-0.029261,-0.133262,-0.057388,-0.020919,-0.152536,-0.100700,-0.076835,0.096953,-0.035623,Anti_microbial


In [ ]:
feature = list(train_features.iloc[0])

In [ ]:
len(feature)

1024

In [ ]:
train_features.to_csv("train_dataframe.csv", encoding='utf-8', index=False)
print("Train_Dataframe Saved")

Train_Dataframe Saved


In [ ]:
test_features.to_csv("test_dataframe.csv", encoding='utf-8', index=False)
print("Test_Dataframe Saved")

Test_Dataframe Saved


In [ ]:
!cp "train_dataframe.csv" "/content/drive/MyDrive/Bayes labs"

**Modeling**

In [ ]:
!pip install pytorch-lightning

     |████████████████████████████████| 925 kB 8.4 MB/s 
     |████████████████████████████████| 636 kB 60.2 MB/s 
     |████████████████████████████████| 829 kB 62.7 MB/s 
     |████████████████████████████████| 125 kB 48.3 MB/s 
     |████████████████████████████████| 282 kB 63.5 MB/s 
     |████████████████████████████████| 1.3 MB 62.2 MB/s 
     |████████████████████████████████| 294 kB 48.5 MB/s 
     |████████████████████████████████| 160 kB 69.0 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 sha256=0c3cc56a2f40d256af57d0a516bb0dc3d6834dfdd678dcc92c91705d63cfe9d1
  Stored in directory: /root/.cache/pip/wheels/56/b0/fe/4410d17b32f1f0c3cf54cdfb2bc04d7b4b8f4ae377e2229ba0
Successfully built future
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: future
    Found existing installation: future 0.16.0
    Uninstalling future-0

In [ ]:
from torch import nn
from torch.nn.functional import relu 
import torch.nn.functional as F
from torch import tensor
from torch.utils.data import DataLoader
import pytorch_lightning as pl
import torch
import pandas as pd
import numpy as np
#from pytorch_lightning.metrics.functional import accuracy
from torchmetrics.functional.classification.accuracy import accuracy

class TorchModel(nn.Module):
    def __init__(self, input_size=1024, hidden_size_1=64,hidden_size_2=128,
                 hidden_size_3=64,hidden_size_4=32,hidden_size_5= 32,
                 #hidden_size_5= 32,hidden_size_6 = 32,hidden_size_7 = 16,
                 output_size=2):
        super(TorchModel, self).__init__()
        self.layer_1 = torch.nn.Linear(input_size,hidden_size_1)
        self.layer_2 = torch.nn.Linear(hidden_size_1, hidden_size_2)
        self.layer_3 = torch.nn.Linear(hidden_size_2, hidden_size_3)
        self.layer_4 = torch.nn.Linear(hidden_size_3, hidden_size_4)
        self.layer_5 = torch.nn.Linear(hidden_size_4, hidden_size_5)
        #self.layer_6 = torch.nn.Linear(hidden_size_5, hidden_size_6)
        #self.layer_7 = torch.nn.Linear(hidden_size_6, hidden_size_7)
        self.output_layer = torch.nn.Linear(hidden_size_5, output_size)
       
    def forward(self,input_data):
        output_of_layer_1 = relu(self.layer_1(input_data))
        output_of_layer_2 = relu(self.layer_2(output_of_layer_1))
        output_of_layer_3 = relu(self.layer_3(output_of_layer_2))
        output_of_layer_4 = relu(self.layer_4(output_of_layer_3))
        output_of_layer_5 = relu(self.layer_5(output_of_layer_4))
        #output_of_layer_6 = relu(self.layer_6(output_of_layer_5))
        #output_of_layer_7 = relu(self.layer_7(output_of_layer_6))
        output = self.output_layer(output_of_layer_5)
        final_output = F.log_softmax(output,dim = 1)
        return final_output


class LightningModel(pl.LightningModule):
    def __init__(self):
        super(LightningModel, self).__init__()
        self.model = TorchModel()
        self.batch_size = 32
        #self.loss = nn.
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(),lr=0.001)

    def cross_entropy_loss(self, output, labels):
        return torch.nn.NLLLoss()(output, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        output = self.model.forward(x)
        train_loss = self.cross_entropy_loss(output, y)
        #predicted_labels = torch.argmax(output, dim=1)
        train_acc = accuracy(torch.argmax(output, dim=1), y)
        #pbar = {"train_acc":acc}
        self.log("train_performance", {"train_acc":train_acc, "train_loss": train_loss},on_step=False, on_epoch=True, prog_bar=True, logger=True)
        #self.log("train_acc", acc, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return train_loss
    
    def test_step(self, test_batch, batch_idx):
        x, y = test_batch
        output = self.model.forward(x)
        test_loss = self.cross_entropy_loss(output, y)
        test_acc = accuracy(torch.argmax(output, dim=1), y)
        self.log("test_performance", {"test_acc":test_acc, "test_loss": test_loss},on_step=True, on_epoch=True, prog_bar=True, logger=True)
        #output = dict({'test_loss': test_loss,'test_acc': test_acc})


        return test_loss


    def prepare_data(self):
        train_data = pd.read_csv(r"/content/drive/MyDrive/Bayes labs/test_dataframe.csv")
        train_data["labels"] = train_data["labels"].map({"Anti_microbial":1 , "non_Anti_microbial" :0})
        train_labels = train_data["labels"]
        train_data = train_data.drop("labels" , axis = 1)
        train_data = torch.tensor(np.array(train_data), dtype=torch.float)
        train_labels=torch.LongTensor(np.array(train_labels))
        self.train_data = []
        for i in range(len(train_data)):
            self.train_data.append((train_data[i], train_labels[i]))
        # self.trn_data = (train_data, train_labels)


        test_data = pd.read_csv(r"/content/drive/MyDrive/Bayes labs/test_dataframe.csv")
        test_data["labels"] = test_data["labels"].map({"Anti_microbial":1 , "non_Anti_microbial" :0})
        test_labels = test_data["labels"]
        test_data = test_data.drop("labels" , axis = 1)
        test_data = (torch.tensor(np.array(test_data), dtype=torch.float))
        test_labels=torch.LongTensor(np.array(test_labels))
        self.test_data = []
        for i in range(len(train_data)):
            self.test_data.append((test_data[i], test_labels[i]))


    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size = self.batch_size)
    
    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size = self.batch_size)


trainer = pl.Trainer(max_epochs=150)
model = LightningModel()
trainer.fit(model)
trainer.test(model,dataloaders = model.test_dataloader())

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name  | Type       | Params
-------------------------------------
0 | model | TorchModel | 85.4 K
-------------------------------------
85.4 K    Trainable params
0         Non-trainable params
85.4 K    Total params
0.342     Total estimated model params size (MB)


Training: -1it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_performance': {'test_acc': tensor(0.9788), 'test_loss': tensor(0.0746)},
 'test_performance_epoch': {'test_acc': tensor(0.9788),
                            'test_loss': tensor(0.0746)}}
--------------------------------------------------------------------------------


[{'test_performance': {'test_acc': tensor(0.9788),
   'test_loss': tensor(0.0746)},
  'test_performance_epoch': {'test_acc': tensor(0.9788),
   'test_loss': tensor(0.0746)}}]

In [ ]:
trainer.save_checkpoint("example.ckpt")


In [ ]:
new_model = MyModel.load_from_checkpoint(checkpoint_path="lightning_logs/version_1/checkpoints/epoch=119-step=3239.ckpt")

In [ ]:
! ls lightning_logs/version_1/checkpoints/'epoch=119-step=3239.ckpt'


'lightning_logs/version_1/checkpoints/epoch=119-step=3239.ckpt'


In [ ]:
trainer = Trainer(default_root_dir="lightning_logs/version_1/checkpoints/epoch=119-step=3239.ckpt")


In [ ]:
new_model = MyModel.load_from_checkpoint(checkpoint_path="lightning_logs/version_1/checkpoints/epoch=119-step=3239.ckpt")
